# Rcpp function tests
Compare function outputs with ```carma_cython_test.ipynb``` equivalent

In [1]:
library(data.table)
library(Rcpp)
library(RcppArmadillo)
library(RcppGSL)
library(CARMA)
library(dplyr)
library(magrittr)
library(devtools)
library(R.utils)
library(Matrix)
library(MASS)
library(glmnet)

Registered S3 methods overwritten by 'RcppGSL':
  method               from         
  predict.fastLm       RcppArmadillo
  print.fastLm         RcppArmadillo
  summary.fastLm       RcppArmadillo
  print.summary.fastLm RcppArmadillo


Attaching package: 'RcppGSL'


The following objects are masked from 'package:RcppArmadillo':

    fastLm, fastLmPure


The following object is masked from 'package:Rcpp':

    LdFlags



Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: usethis

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.25.0 (2022-06-12 02:20:02 UTC) successfully loaded. See ?R.oo for help.


Attachi

In [7]:
index_vec_input <- c(1, 2)
Sigma <- matrix(c(1, 0.5, 0.5, 1), nrow = 2)  
z <- c(1, 2)  
tau <- 0.5
p_S <- 1
y_sigma <- 1

result <- CARMA::Normal_fixed_sigma_marginal(index_vec_input, Sigma, z, tau, p_S, y_sigma)
print(result)

[1] 0.7840272


In [2]:
set.gamma.func<-function(input.S,condition.index=NULL){
    set.gamma.func.base<-function(S){
        add.function<-function(y){results<-(apply(as.matrix(S_sub),1,function(x){return(sort(c(x,y)))}))
        return(t(results))
        }
        
        set.gamma<-list()
        for(i in 1:3){
        set.gamma[[i]]<-c()
        }
        
        #set of gamma-
        if(length(S)==0){
        S_sub<-(1:p)
        set.gamma[[1]]<-c()
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(c(x,S))})
        set.gamma[[2]]<-as.matrix(set.gamma[[2]])
        set.gamma[[3]]<-c()
        }
        if(length(S)>1){
        S_sub<-(1:p)[-S]
        set.gamma[[1]]<-t(combn(S,length(S)-1))
        if(length(S)>2){
            set.gamma[[1]]<-t(apply(as.matrix(set.gamma[[1]]),1,sort))
        }
        #set of gamma+
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(sort(c(x,S)))})
        set.gamma[[2]]<-t(set.gamma[[2]])
        #set of gamma=
        set.gamma[[3]]<-add.function(set.gamma[[1]][1,])
        for(i in 2:nrow(set.gamma[[1]])){
            set.gamma[[3]]<-rbind(set.gamma[[3]],add.function(set.gamma[[1]][i,]))  
        }
        }
        if(length(S)==1){
        S_sub<-(1:p)[-S]
        set.gamma[[1]]<-t(combn(S,length(S)-1))
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(sort(c(x,S)))})
        set.gamma[[2]]<-t(set.gamma[[2]])
        set.gamma[[3]]<-t(add.function(set.gamma[[1]][1,]))
        }
        return(set.gamma)
    }
    set.gamma.func.conditional<-function(input.S,condition.index){
        add.function<-function(y){results<-(apply(as.matrix(S_sub),1,function(x){return(sort(c(x,y)))}))
        return(t(results))
        }
        
        set.gamma<-list()
        for(i in 1:3){
        set.gamma[[i]]<-c()
        }
        S=input.S[-match(condition.index,input.S)]
        
        #set of gamma-
        if(length(S)==0){
        S=integer(0)
        S_sub<-(1:p)[-condition.index]
        set.gamma[[1]]<-c()
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(c(x,S))})
        set.gamma[[2]]<-as.matrix(set.gamma[[2]])
        set.gamma[[3]]<-c()
        }
        if(length(S)==1){
        S_sub<-(1:p)[-input.S]
        set.gamma[[1]]<-t(combn(S,length(S)-1))
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(sort(c(x,S)))})
        set.gamma[[2]]<-as.matrix((t(set.gamma[[2]])))
        set.gamma[[3]]<-t(add.function(set.gamma[[1]][1,]))
        }
        if(length(S)>1){
        S_sub<-(1:p)[-input.S]
        if(length(S)>2){
            set.gamma[[1]]<-t(combn(S,length(S)-1))
            set.gamma[[1]]<-t(apply(as.matrix(set.gamma[[1]]),1,sort))
        }else{
            set.gamma[[1]]<-t(combn(S,length(S)-1))
        }
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(sort(c(x,S)))})
        set.gamma[[2]]<-as.matrix(t(set.gamma[[2]]))
        set.gamma[[3]]<-add.function(set.gamma[[1]][1,])
        factorial(3)
        for(i in 2:nrow(set.gamma[[1]])){
            set.gamma[[3]]<-rbind(set.gamma[[3]],add.function(set.gamma[[1]][i,]))  
        }
        }
        
        return(set.gamma)
    }
    
    if(is.null(condition.index)){
    results<-set.gamma.func.base(input.S)
    }else{
    results<-set.gamma.func.conditional(input.S,condition.index)
    }
    return(results)
}

# Define the value of p
p <- 481

# Example usage with input.S
input.S <- c()
result <- set.gamma.func(input.S)
print(result)

# Example usage with input.S and condition.index
input.S <- c()
condition.index <- 2
result_with_condition <- set.gamma.func(input.S, condition.index)
print(result_with_condition)

[[1]]
NULL

[[2]]
       [,1]
  [1,]    1
  [2,]    2
  [3,]    3
  [4,]    4
  [5,]    5
  [6,]    6
  [7,]    7
  [8,]    8
  [9,]    9
 [10,]   10
 [11,]   11
 [12,]   12
 [13,]   13
 [14,]   14
 [15,]   15
 [16,]   16
 [17,]   17
 [18,]   18
 [19,]   19
 [20,]   20
 [21,]   21
 [22,]   22
 [23,]   23
 [24,]   24
 [25,]   25
 [26,]   26
 [27,]   27
 [28,]   28
 [29,]   29
 [30,]   30
 [31,]   31
 [32,]   32
 [33,]   33
 [34,]   34
 [35,]   35
 [36,]   36
 [37,]   37
 [38,]   38
 [39,]   39
 [40,]   40
 [41,]   41
 [42,]   42
 [43,]   43
 [44,]   44
 [45,]   45
 [46,]   46
 [47,]   47
 [48,]   48
 [49,]   49
 [50,]   50
 [51,]   51
 [52,]   52
 [53,]   53
 [54,]   54
 [55,]   55
 [56,]   56
 [57,]   57
 [58,]   58
 [59,]   59
 [60,]   60
 [61,]   61
 [62,]   62
 [63,]   63
 [64,]   64
 [65,]   65
 [66,]   66
 [67,]   67
 [68,]   68
 [69,]   69
 [70,]   70
 [71,]   71
 [72,]   72
 [73,]   73
 [74,]   74
 [75,]   75
 [76,]   76
 [77,]   77
 [78,]   78
 [79,]   79
 [80,]   80
 [81,]   8

In [15]:
set.gamma <- set.gamma.func(input.S)
Sigma <- as.matrix(fread("/Users/hn9/Documents/GitHub/carmapy/sigma.csv"))
z <- as.vector(fread("/Users/hn9/Documents/GitHub/carmapy/z.csv")[[1]])
tau.sample <- 0.04
p_S <- 481
y_var <- 1

marginal_likelihood=Normal_fixed_sigma_marginal
set.gamma.margin<-list()
set.gamma.margin[[2]]<-apply(set.gamma[[2]],1,marginal_likelihood,Sigma=Sigma,z=z,tau=tau.sample,p_S=p_S,y_sigma=y.var)

In [16]:
set.gamma.margin[[2]]

[1] -783.17347 -783.57202 -783.01878 -782.25610 -783.55544 -783.14576
  [7] -782.20691 -782.74109 -779.60721 -782.02093 -783.26273 -768.58490
 [13] -780.54064 -783.15844 -781.89475 -783.21730 -782.14623 -751.08473
 [19] -781.34791 -783.10244 -783.24294 -783.04327 -783.47561 -783.33956
 [25] -771.91650 -783.32299 -783.26126 -783.49529 -782.97060 -768.02458
 [31] -783.26856 -783.16257 -783.51570 -783.29186 -783.51484 -781.03713
 [37] -783.51270 -782.48489 -774.76529 -783.28451 -783.54604 -783.30711
 [43] -783.44918 -783.47335 -783.51755 -768.57988 -779.46772 -779.01700
 [49] -779.06313 -781.77731 -783.40701 -783.36215 -782.26626 -783.17347
 [55] -783.01108 -783.14338 -782.43641 -766.18952 -765.65993 -780.33098
 [61] -781.29788 -783.34005 -782.55745 -738.53001 -782.47684 -782.44928
 [67] -738.88053 -782.55745 -738.59065 -779.53367 -783.53686 -777.68279
 [73] -782.51914 -783.19235 -783.37296 -782.88219 -783.11697 -782.25846
 [79] -783.49356 -780.39772 -772.14644 -783.51991 -782.41899 -783.53078
 [85] -783.28362 -783.08082 -783.57146 -783.53930 -783.54560 -783.45539
 [91] -783.56471 -783.52670 -782.56727 -782.47978 -782.72479 -783.55346
 [97] -783.34242 -782.41784 -780.15341 -780.51492 -783.52255 -782.33904
[103] -783.50734 -781.99539 -782.98425 -782.97567 -783.09145 -783.46320
[109] -782.44453 -783.37353 -783.47153 -783.56650 -783.48194 -783.45606
[115] -783.52072 -783.38162 -783.49529 -783.28963 -783.40947 -783.10877
[121] -783.05585 -750.96641 -741.01224 -750.38157 -747.29508 -749.19628
[127] -721.27789 -783.56628 -783.25858 -718.29666 -756.47168 -743.43803
[133] -741.96338 -743.68359 -742.23401 -741.44957 -755.73443 -755.24722
[139] -753.25762 -647.24455 -768.39594 -775.82988 -756.27970 -771.55299
[145] -771.81379 -755.76451 -771.37753 -771.81379 -771.81379 -771.88800
[151] -782.81771 -776.48775 -673.03154 -783.09145 -782.12976 -781.92349
[157] -774.81341 -783.45321 -783.54127 -783.32234 -779.28824 -762.42517
[163] -765.16152 -783.53006 -689.34145 -740.49721 -782.91784 -649.49049
[169] -782.81624 -419.32913 -404.92133 -776.67775 -737.49828 -782.64904
[175] -780.50297 -781.31527 -780.45264 -781.28692 -669.11842 -778.90221
[181] -766.04192 -742.48438 -777.56322 -783.13832 -781.68594 -781.00602
[187] -765.17966 -717.62618 -721.09145 -497.46492 -500.49285 -777.48634
[193] -649.10261 -716.22008 -738.28176 -623.79880 -742.08456 -761.75249
[199] -634.28995 -782.14774 -749.19628 -774.86974 -781.75307 -772.91090
[205] -776.43058 -767.10867 -782.25846 -772.34091 -766.80451 -781.32126
[211] -710.72396 -745.35983 -769.19374 -772.84748 -573.43341 -780.08947
[217] -679.51140 -782.49049 -748.77374 -684.55891 -781.03492 -563.18804
[223] -782.71752 -782.48347 -749.13501 -750.54869 -565.05966   16.01036
[229] -533.17914 -507.67383 -287.34065 -783.23045 -733.04664 -782.92879
[235] -782.92879 -733.04664 -727.74013 -733.51774 -371.96279 -736.19629
[241] -729.65423 -774.57276 -771.74078 -776.23726 -781.65581 -778.93233
[247] -778.46512 -773.29875 -780.54629 -771.36371 -584.96374 -781.96677
[253] -402.02981 -758.54042  855.89771 -781.04194 -705.99078 -758.87938
[259]  184.11827 -737.11121 -446.95710 -372.70054 -752.54604 -709.81164
[265] -782.10193 -744.34566 -783.56791 -782.33590 -534.50626 -768.58490
[271] -759.66510 -783.41658 -762.75664 -497.75156 -777.77766 -783.44463
[277] -783.18501 -767.85350 -769.23164 -774.86657 -775.50668 -783.50461
[283] -771.18601 -774.52088 -774.56719 -775.26064 -783.55014 -773.21263
[289] -772.67042 -777.13070 -765.70535 -781.04901 -772.69398 -781.12848
[295] -773.18283 -781.08891 -766.86302 -783.36909 -776.46025 -776.55139
[301] -783.29553 -783.21493 -780.25942 -781.36460 -782.27272 -782.35864
[307] -783.42678 -782.11540 -781.86696 -782.65221 -782.56140 -780.22670
[313] -782.47863 -770.21564 -776.13005 -769.77076 -770.58263 -771.69678
[319] -781.00962 -771.03609 -780.88983 -781.34460 -781.12915 -782.25610
[325] -783.03655 -773.87742 -773.75325 -773.95872 -773.21471 -773.13033
[331] -773.72244 -772.76711 -772.61687 -77

In [6]:
p <- 2799
input.S <- c()
set.gamma <- set.gamma.func(input.S)
Sigma <- as.matrix(fread("/Users/hn9/Documents/GitHub/carmapy/carma_sigma.csv"))
z <- as.vector(fread("/Users/hn9/Documents/GitHub/carmapy/carma_z.csv")[[1]])
tau.sample <- 0.04
p_S <- 1
y_var <- 1

marginal_likelihood=Normal_fixed_sigma_marginal
set.gamma.margin<-list()
set.gamma.margin[[2]]<-apply(set.gamma[[2]],1,marginal_likelihood,Sigma=Sigma,z=z,tau=tau.sample,p_S=p_S,y_sigma=y_var)

In [7]:
set.gamma.margin[[2]]

[1] -1.520620135 -1.483421980 -1.415697081 -1.483421980 -1.626600709
   [6] -1.628576334 -1.558757559 -0.696465390 -0.507370234 -0.507370234
  [11] -0.960960284 -0.507370234 -0.507370234  0.297083779 -1.283038601
  [16] -1.398375765 -0.412180286 -1.250420748 -1.415697081 -1.584182160
  [21] -1.424339169 -1.332467948  0.010268637 -1.345450864 -1.584182160
  [26] -1.584182160 -1.584182160 -1.232919069 -1.584182160 -1.606957948
  [31] -1.584182160 -1.584182160 -1.628720935 -1.584182160 -1.584182160
  [36] -1.584182160 -1.584182160 -1.345450864 -1.584182160 -1.584182160
  [41] -1.584182160 -1.541173336 -1.332467948 -1.584182160 -1.584182160
  [46] -1.621611288 -1.584182160 -1.415697081 -1.407402369 -1.388262101
  [51] -1.584182160 -1.584182160 -1.584182160 -1.584182160 -1.445764274
  [56] -1.584182160 -1.584182160 -1.584182160 -0.961325697 -1.332467948
  [61] -1.345450864 -1.584182160 -1.613539023 -1.578099731 -1.628997242
  [66] -1.445764274 -1.332467948 -1.442718812 -1.628842908 -1.628842908
  [71] -1.628842908 -1.628842908 -1.577499596 -1.628842908 -1.628842908
  [76] -1.628842908 -1.628842908 -1.445764274 -1.628842908 -1.628842908
  [81] -1.415697081 -1.628842908 -1.627863628 -1.628842908 -1.503661439
  [86] -1.370855167 -1.600862171 -1.618423652 -1.618423652 -1.618423652
  [91] -0.802581622 -1.487952756 -1.146937283 -1.618423652 -1.618423652
  [96] -1.618423652 -1.564540915 -1.513783748 -1.542683778 -1.403491336
 [101] -1.541796263 -1.600846747 -1.627761956 -1.627761956 -1.626438408
 [106] -1.627761956 -1.553464347 -1.542683778 -1.498233631 -1.436245716
 [111] -1.627761956 -1.483968681 -1.542252341 -1.627761956 -1.526558289
 [116] -0.014300573 -1.627761956 -1.536711475 -1.423909492 -1.571819466
 [121] -1.357669509 -1.344775744 -1.541802435 -1.473657585 -1.617042143
 [126] -1.541802435 -1.473657585 -1.617042143 -1.473657585 -1.526558289
 [131] -1.357669509 -1.617042143 -1.621877700 -1.617042143 -0.724192623
 [136] -1.524326177 -1.590404432 -1.541802435 -1.629014852 -1.123713932
 [141] -1.610119304 -1.590404432 -1.511532533 -1.123713932 -1.439239778
 [146] -1.590404432 -1.623416481 -1.245463088 -1.590404432 -0.700879248
 [151] -1.439239778 -1.565510509 -0.700879248 -1.526973932 -1.182076306
 [156] -1.229051334 -1.487540423 -0.963233273 -1.586711259 -0.985955880
 [161] -1.547100423 -1.032997847 -0.961219331 -1.555125081 -1.547100423
 [166]  0.650789881 -1.273377169 -1.586711259 -1.428691005 -1.586711259
 [171] -1.581500152 -1.273377169 -1.581500152 -1.083042987 -1.616335922
 [176] -1.581500152 -1.547100423 -1.581500152 -1.352913950 -1.613479187
 [181] -1.348820265  0.650789881 -1.348820265 -1.348820265 -1.543827055
 [186] -1.606390994 -1.625453793 -1.460545843 -1.460545843 -1.622598196
 [191] -1.374733573 -1.374733573 -0.776365207 -1.557876428 -1.620451159
 [196] -1.620451159 -0.254011173 -1.538133814 -1.487318981 -1.622848039
 [201] -1.109890269 -1.596928454 -1.375489615 -1.454172784 -1.375489615
 [206] -1.627849756 -1.239213920 -1.628986196 -0.977545827 -1.239213920
 [211] -1.239213920 -1.239213920 -1.109798932 -0.644840877 -1.599309995
 [216]  0.055322199 -1.434316603 -1.516716562 -1.508206710 -1.176604773
 [221] -1.586545417 -0.607059989 -1.334943238 -1.510988894 -0.484247573
 [226] -1.617416213 -1.456140833 -1.397644939 -1.305562426 -1.397644939
 [231] -1.600455220  1.957292921  1.957292921 -1.562731774 -1.562731774
 [236]  0.448467984  4.018925929  0.284749256 -1.546714984 -1.626911879
 [241] -1.562731774 -1.562731774 -0.106223522 -1.562731774  5.053877842
 [246] -1.264139029  5.053877842 -1.614084547 -0.518616279 -1.296087976
 [251] -1.623435225  1.318373052 -1.597516941 -0.310636450 -1.627635128
 [256] -1.623435225  6.143428150 -1.628348978 -1.508208580 -0.427906134
 [261] -1.622246307 -1.628351552 -1.628642653 -1.334933007 -0.832261298
 [266] -1.319430920 -0.286771958 -1.538883067  6.272386411 -1.564755981
 [271] -0.286771958 -0.832261298 -0.987794565 -1.521829709 -1.555470273
 [276] -1.618969196 -1.521829709 -0.2867719

In [ ]:
# Entire locus entered (output doesn't seem to match at scale)
index_vec_input <- 1:481
Sigma <- as.matrix(fread("/Users/hn9/Documents/GitHub/carmapy/sigma.csv"))
z <- as.vector(fread("/Users/hn9/Documents/GitHub/carmapy/z.csv")[[1]])
tau <- 0.04
p_S <- 481
y_sigma <- 1

result <- CARMA::Normal_fixed_sigma_marginal(index_vec_input, Sigma, z, tau, p_S, y_sigma)
print(result)

In [3]:
index_vec_input <- c(1, 2) 
Sigma <- matrix(c(1, 0.5, 0.5, 1), nrow = 2)
z <- c(1, 2)
tau <- 0.5
p_S <- 2
y_sigma <- 1

result <- outlier_Normal_fixed_sigma_marginal(index_vec_input, Sigma, z, tau, p_S, y_sigma)
print(result)

[1] -0.5228256


In [4]:
index_vec_input <- c(1, 2)
Sigma <- matrix(c(1, 0.5, 0.5, 1), nrow = 2)
z <- c(1, 2)  
tau <- 0.5
p_S <- 2
y_sigma <- 1

result <- CARMA::ind_Normal_fixed_sigma_marginal(index_vec_input, Sigma, z, tau, p_S, y_sigma)
print(result)

[1] 0.3352792
